In [1]:
import sys, os

In [2]:
sys.path.append(os.path.abspath('../autoencoders'))

In [3]:
# "Magic" commands for automatic reloading of module, perfect for prototyping
%reload_ext autoreload
%autoreload 2

import wasserstein
import proteins

In [4]:
import torch

In [ ]:
model = proteins.WassersteinAutoEncoder(ksi=10.)
model.load_state_dict(torch.load(MODELS_PATH + 'wae.weights'))